# 複数の関数パラメータを取得し、実行するデモ

簡単なフロー：

1. タスクを指示する（例：テーブルを経由してキッチンに移動して）
2. ChatCompletion実行
3. 応答の `finish_reason` が `function_call` の場合、パラメータを保持して再度そのままChatCompletionを実行する
4. 応答の `finish_reason` が `stop` になったら、保持したパラメータをもとに関数を実行し、ユーザーに制御を渡す

In [1]:
%pip install openai
%pip install tenacity

     ---------------------------------------- 73.6/73.6 kB 4.0 MB/s eta 0:00:00
  Using cached tqdm-4.65.0-py3-none-any.whl (77 kB)
  Using cached aiohttp-3.8.4-cp310-cp310-win_amd64.whl (319 kB)
  Using cached requests-2.31.0-py3-none-any.whl (62 kB)
  Using cached urllib3-2.0.3-py3-none-any.whl (123 kB)
  Using cached charset_normalizer-3.2.0-cp310-cp310-win_amd64.whl (96 kB)
  Using cached certifi-2023.5.7-py3-none-any.whl (156 kB)
  Using cached idna-3.4-py3-none-any.whl (61 kB)
  Using cached attrs-23.1.0-py3-none-any.whl (61 kB)
  Using cached yarl-1.9.2-cp310-cp310-win_amd64.whl (61 kB)
  Using cached frozenlist-1.3.3-cp310-cp310-win_amd64.whl (33 kB)
  Using cached multidict-6.0.4-cp310-cp310-win_amd64.whl (28 kB)
  Using cached async_timeout-4.0.2-py3-none-any.whl (5.8 kB)
  Using cached aiosignal-1.3.1-py3-none-any.whl (7.6 kB)
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


  Using cached tenacity-8.2.2-py3-none-any.whl (24 kB)
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import openai
from tenacity import retry, wait_random_exponential, stop_after_attempt
import requests
import time
import json

GPT_MODEL = "gpt-4-0613"

## 関数の定義

In [3]:
def move_to_object(name: str):
    print("move to object: " + name)

def set_speed(speed: float):
    print("set_speed: " + speed)

## ChatCompletionの定義

In [4]:
@retry(wait=wait_random_exponential(min=1, max=40), stop=stop_after_attempt(3))
def chat_completion_request(
    messages: dict,
    functions: list = None,
    function_call: dict = None,
    model: str = GPT_MODEL
):
    headers = {
        "Content-Type": "application/json",
        "Authorization": "Bearer " + openai.api_key,
    }
    json_data = {"model": model, "messages": messages}
    if functions is not None:
        json_data.update({"functions": functions})
    if function_call is not None:
        json_data.update({"function_call": function_call})
    try:
        response = requests.post(
            "https://api.openai.com/v1/chat/completions",
            headers=headers,
            json=json_data,
        )
        return response
    except Exception as e:
        print("Unable to generate ChatCompletion response")
        print(f"Exception: {e}")
        return e

## 関数実行処理の定義

In [5]:
def execute_function_call(assistant_message: dict):
    try:
        parsed_output = json.loads(
            assistant_message["function_call"]["arguments"]
        )
        arguments = ", ".join(str(arg) for arg in parsed_output.values())
        method = f"{assistant_message['function_call']['name']}('{arguments}')"
        eval(method)
    except Exception as e:
        print(e)

## functionsの定義

In [6]:
functions = [
    {
        "name": "move_to_object",
        "description": "Move to target position",
        "parameters": {
            "type": "object",
            "properties": {
                "name": {
                    "type": "string",
                    "description": "target object name"
                }
            },
            "required": ["name"]
        },
    },
    {
        "name": "set_speed",
        "description": "Set move speed",
        "parameters": {
            "type": "object",
            "properties": {
                "speed": {
                    "type": "integer",
                    "description": "The number of speed",
                }
            },
            "required": ["speed"]
        },
    },
]

## システムメッセージと最初のユーザーメッセージ

「テーブルを経由してキッチンに移動して」

In [7]:
messages = []
messages.append(
    {
        "role": "system",
        "content": "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."
    }
)
messages.append(
    {
        "role": "user",
        "content": "Move to the kitchen via the table."
    }
)
chat_response = chat_completion_request(
    messages, functions=functions
)
assistant_message = chat_response.json()["choices"][0]["message"]
messages.append(assistant_message)
print(assistant_message)

{'role': 'assistant', 'content': 'To make sure I understand your instructions correctly, would you like me to:\n\n1. Move to the table first, then proceed to the kitchen?\nor\n2. Navigate directly to the kitchen, passing by the table?'}


指示が曖昧だと、「テーブルに移動してからキッチンに移動するか、テーブルを取って直接キッチンに行くか、どちらですか？」と聞き返してくる

「テーブルに移動してからキッチンに移動して」と指示

In [13]:
messages.append(
    {
        "role": "user",
        "content": "First we move to the table and then proceed to the kitchen."
    }
)
chat_response = chat_completion_request(
    messages, functions=functions
)
print(chat_response.json())

{'id': 'chatcmpl-7b2Ja4hy28bc2i9U1NgnXgZJO6NEY', 'object': 'chat.completion', 'created': 1689061390, 'model': 'gpt-4-0613', 'choices': [{'index': 0, 'message': {'role': 'assistant', 'content': None, 'function_call': {'name': 'move_to_object', 'arguments': '{\n  "name": "table"\n}'}}, 'finish_reason': 'function_call'}], 'usage': {'prompt_tokens': 171, 'completion_tokens': 16, 'total_tokens': 187}}


以下のようにパラメータを生成してくれた。

`'function_call': {'name': 'move_to_object', 'arguments': '{\n  "name": "table"\n}'}}`

キッチンへ移動してほしいので、この応答を`messages`にふくめてもう一度`ChatCompletion`を実行

In [14]:
assistant_message = chat_response.json()["choices"][0]["message"]
messages.append(assistant_message)

chat_response = chat_completion_request(
    messages, functions=functions
)
print(chat_response.json())

{'id': 'chatcmpl-7b2MmGW6T7gos6s0rHx3YfQzjJ9NZ', 'object': 'chat.completion', 'created': 1689061588, 'model': 'gpt-4-0613', 'choices': [{'index': 0, 'message': {'role': 'assistant', 'content': None, 'function_call': {'name': 'move_to_object', 'arguments': '{\n  "name": "kitchen"\n}'}}, 'finish_reason': 'function_call'}], 'usage': {'prompt_tokens': 190, 'completion_tokens': 17, 'total_tokens': 207}}


キッチンへ移動するパラメータを生成してくれた。

```
'function_call': {'name': 'move_to_object', 'arguments': '{\n  "name": "kitchen"\n}'}}
```

この応答を含めて更に`ChatCompletion`を実行すると、応答メッセージを返してくれる

In [16]:
# assistant_message = chat_response.json()["choices"][0]["message"]
# messages.append(assistant_message)

chat_response = chat_completion_request(
    messages, functions=functions
)
print(chat_response.json())

{'id': 'chatcmpl-7b2P0tyvxUKjRuplgC59DN4Wg5jRZ', 'object': 'chat.completion', 'created': 1689061726, 'model': 'gpt-4-0613', 'choices': [{'index': 0, 'message': {'role': 'assistant', 'content': 'Alright, I have moved to the table first and then proceeded to the kitchen.'}, 'finish_reason': 'stop'}], 'usage': {'prompt_tokens': 210, 'completion_tokens': 17, 'total_tokens': 227}}


応答：

```
'content': 'Alright, I have moved to the table first and then proceeded to the kitchen.'
```

これらを対話的に実行できるようにしたスクリプトはこちら

[multiple_function_calling_demo.py](./multiple_function_calling_demo.py)